In [4]:

import numpy as np 
import sciann as sn 
import matplotlib.pyplot as plt 
import scipy.io
from sciann.utils.math import diff, sign, sin


R = 0.1
Vc = 1
Vh = 1

x = sn.Variable("x", dtype='float64')
y = sn.Variable("y", dtype='float64')
t = sn.Variable("t", dtype='float64')

theta_w = sn.Functional("theta_w", [t], 4*[20], 'tanh')
theta_h = sn.Functional("theta_h", [x, t], 8*[20], 'tanh')
theta_c = sn.Functional("theta_c", [y, t], 8*[20], 'tanh')

theta_w_t = sn.diff(theta_w, t)
theta_h_t = sn.diff(theta_h, t); theta_h_x = sn.diff(theta_h, x); 
theta_c_t = sn.diff(theta_c, t); theta_c_y = sn.diff(theta_c, y); 

# define governing equations
L1 = theta_h_t - R/Vh*(theta_w-theta_h-theta_h_x)
L2 = theta_c_t - 1/Vc*(theta_w-theta_c-theta_c_y)
L3 = theta_w_t - theta_c - R*theta_h + (1+R)*theta_w
# initial conditions
TOL = 0.001
C1 = (1-sign(t - TOL)) * (theta_w)
C2 = (1-sign(t - TOL)) * (theta_h)
C3 = (1-sign(t - TOL)) * (theta_c)
# boundary conditions
C4 = (1-sign(x - TOL)) * (theta_h - 1.)
C5 = (1-sign(y - TOL)) * (theta_c_y)
C6 = (1+sign(x - ( 1-TOL))) * (theta_h_x)
C7 = (1+sign(y - ( 1-TOL))) * (theta_c)

# Define the optimization model (set of inputs and constraints)
m = sn.SciModel([x, y, t], [L1, L2, L3, C1, C2, C3, C4, C5, C6, C7])
x_data, y_data, t_data = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100),
    np.linspace(0, 1, 100),
)

h = m.train([x_data, y_data, t_data], 10*['zero'], 
    epochs=10000,
    shuffle=True,
    batch_size=100,
    verbose=1,
    learning_rate=1e-3,
    reduce_lr_after=100,
    stop_loss_value=1e-8
    )

x_test, y_test, t_test = np.meshgrid(
    np.linspace(0, 1, 200), 
    np.linspace(0, 1, 200), 
    np.linspace(0, 1, 200)
)
theta_h_pred = theta_h.eval(m, [x_test, y_test, t_test])
theta_c_pred = theta_c.eval(m, [x_test, y_test, t_test])
theta_w_pred = theta_w.eval(m, [x_test, y_test, t_test])

fig = plt.figure(figsize=(3, 4))
plt.pcolor(x_test, t_test, theta_h_pred, cmap='seismic')
plt.xlabel('x')
plt.ylabel('t')

plt.colorbar()

# model.save_weights('trained-navier-stokes.hdf5')


---------------------- SCIANN 0.6.0.4 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 



ModuleNotFoundError: No module named 'matplotlib'